# Scraping datos resultados anuales investing

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
from bs4 import BeautifulSoup




def get_resultados(tk):
    dfs = []
    # Configurar el navegador
    # chrome_options = Options()
    # chrome_options.add_argument("--headless")
    driver = webdriver.Chrome()
    try:
        # Abrir la URL
        url = f"https://es.investing.com/equities/{tk}-income-statement"
        driver.get(url)

        # Esperar a que al menos una tabla esté presente
        
        wait = WebDriverWait(driver, 5)
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

        # Obtener el HTML de la página
        page_html = driver.page_source

        # Parsear el HTML de la página con BeautifulSoup
        soup = BeautifulSoup(page_html, "html.parser")

        # Encontrar todas las tablas en la página
        tables = soup.find_all("table")

        # Procesar cada tabla
        for i, table in enumerate(tables):
            # Extraer las filas de la tabla
            rows = table.find_all("tr")
            data = []
            for row in rows:
                cells = row.find_all(["td", "th"])  # Buscar celdas (td) y encabezados (th)
                cell_texts = [cell.get_text(strip=True) for cell in cells]
                data.append(cell_texts)

            # Convertir los datos en un DataFrame de Pandas
            df = pd.DataFrame(data)
            dfs.append(df)
        print("exito al procesar!")

    except TimeoutException:
        print("Error: No se encontraron tablas en la página.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Cerrar el navegador
        driver.quit()
        time.sleep(5)
    return dfs

#### Procesamiento de datos capturados

In [66]:
def procesar_tab(df):
  data = pd.DataFrame()
   
  data = df[1].replace("aa.aa",None).reset_index(drop=True)
  data = data.drop(data[data[1]==""].index,axis=0)
  data = data[data[1].notna()]
  data = data.drop(columns=[0])
  data.columns = data.iloc[0]
  data.drop(0,inplace=True,axis=0)
  data = data.dropna(axis=1,how="all")
  data.rename(columns={data.columns[0]:"indicador"},inplace=True)

  return data.set_index("indicador")

### Lista de Empresas

In [70]:
companies = [
  "apple-computer-inc",
  "microsoft-corp",
  "google-inc",
  "tesla-motors",
  "visa-inc",
  "berkshire-hathaway-inc",
  "johnson-johnson",
  "pfizer",
  "amazon-com-inc",
  "disney",
  "nike",
  "procter-gamble",
  "coca-cola-co",
  "chevron",
  "3m-co"
]

In [71]:
import os 
os.makedirs("resultados",exist_ok=True)
for company in companies:
  print(f"Procesando {company}")
  dfs = get_resultados(company)
  if len(dfs) > 1:
    df = procesar_tab(dfs)
    df["company"] = company
    df.to_csv(f"resultados/{company}.csv",encoding="utf-8-sig")
    print(f"Guardado {company}.csv")
  else:
    print(f"Error al procesar {company}")

Procesando apple-computer-inc
exito al procesar!
Guardado apple-computer-inc.csv
Procesando microsoft-corp
exito al procesar!
Guardado microsoft-corp.csv
Procesando google-inc
exito al procesar!
Guardado google-inc.csv
Procesando tesla-motors
exito al procesar!
Guardado tesla-motors.csv
Procesando visa-inc
exito al procesar!
Guardado visa-inc.csv
Procesando berkshire-hathaway-inc
exito al procesar!
Guardado berkshire-hathaway-inc.csv
Procesando johnson-johnson
exito al procesar!
Guardado johnson-johnson.csv
Procesando pfizer
exito al procesar!
Guardado pfizer.csv
Procesando amazon-com-inc
exito al procesar!
Guardado amazon-com-inc.csv
Procesando disney
exito al procesar!
Guardado disney.csv
Procesando nike
exito al procesar!
Guardado nike.csv
Procesando procter-gamble
exito al procesar!
Guardado procter-gamble.csv
Procesando coca-cola-co
exito al procesar!
Guardado coca-cola-co.csv
Procesando chevron
exito al procesar!
Guardado chevron.csv
Procesando 3m-co
exito al procesar!
Guardado 3